In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import glob
import seaborn as sns
import statsmodels.api as sm
import sklearn.preprocessing

In [2]:
# Read in rb stats files
wrStatsFiles = [filename for filename in glob.glob("C:/Users/Michael/Desktop/Springboard/Capstone2/csvFiles/DFS_Data/WR/stats/*.csv")]
wrStatsFiles[0]

'C:/Users/Michael/Desktop/Springboard/Capstone2/csvFiles/DFS_Data/WR/stats\\WR2011.csv'

In [3]:
# Extract season and fix playoff weeks to match other tables

dfs = []
dfsPlayoffs = []

for i in range(int(len(wrStatsFiles)/2)):
    df = pd.read_csv(wrStatsFiles[i])
    df['season'] = wrStatsFiles[i][-8:-4]
    dfs.append(df)

# second half of list has playoff games, need to fix their weeks, then combine with first table    
for i in range(int(len(wrStatsFiles)/2),int(len(wrStatsFiles)/2)*2):
    dfPlayoffs = pd.read_csv(wrStatsFiles[i])
    dfPlayoffs['season'] = wrStatsFiles[i][-8:-4]
    dfsPlayoffs.append(dfPlayoffs)

    
for i in range(len(dfsPlayoffs)):
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 1, 'Week'] = 18
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 2, 'Week'] = 19
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 3, 'Week'] = 20
    dfsPlayoffs[i].loc[dfsPlayoffs[i].Week == 4, 'Week'] = 21


# dfsPlayoffs[-1].head()

wr = pd.concat(dfs,axis=0)
wr = wr.append(dfsPlayoffs)
wr.head()

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,ReceivingYardsPerReception,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGameFanDuel,FantasyPointsFanDuel,season
0,1,Wes Welker,NE,WR,3,BUF,20,16,80.0,217,...,13.6,1,19,19.0,0,0,0,43.6,43.6,2011
1,2,Jordy Nelson,GB,WR,17,DET,15,9,60.0,162,...,18.0,0,0,0.0,0,0,0,38.7,38.7,2011
2,3,Calvin Johnson,DET,WR,15,OAK,14,9,64.3,214,...,23.8,0,0,0.0,0,0,0,37.9,37.9,2011
3,4,Torrey Smith,BAL,WR,3,LAR,8,5,62.5,152,...,30.4,1,10,10.0,0,0,0,36.7,36.7,2011
4,5,Miles Austin,DAL,WR,2,SF,15,9,60.0,143,...,15.9,1,-2,-2.0,0,1,0,36.6,36.6,2011


In [4]:
# Pull data from other workbooks
awayMatchup = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//awayMatchup.csv',index_col=0)
homeMatchup = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//homeMatchup.csv',index_col=0)
oddsTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//oddsTable.csv',index_col=0)
finalTeamGameDataTable = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//teamGameData.csv',index_col=0)
rollAvgGameData = pd.read_csv('C://Users//Michael//Desktop//Springboard//Capstone2//csvFiles//rollAvgGameData.csv',index_col=0)
%store -r awayTable
%store -r homeTable

In [5]:
# Clean up the Team column to match the team column in awayTable and homeTable
teams1 = set(wr['Team'])
teams1.remove('SF')
teams2 = set(awayTable['team'])
teams2.remove('San Francisco 49ers')
t1List = list(teams1)
t2List = list(teams2)
t1List.sort()
t2List.sort()
zipTeams = zip(t1List, t2List)
teamDict = dict(list(zipTeams))
teamDict['SF'] = 'San Francisco 49ers'
teamDict

{'ARI': 'Arizona Cardinals',
 'ATL': 'Atlanta Falcons',
 'BAL': 'Baltimore Ravens',
 'BUF': 'Buffalo Bills',
 'CAR': 'Carolina Panthers',
 'CHI': 'Chicago Bears',
 'CIN': 'Cincinnati Bengals',
 'CLE': 'Cleveland Browns',
 'DAL': 'Dallas Cowboys',
 'DEN': 'Denver Broncos',
 'DET': 'Detroit Lions',
 'GB': 'Green Bay Packers',
 'HOU': 'Houston Texans',
 'IND': 'Indianapolis Colts',
 'JAX': 'Jacksonville Jaguars',
 'KC': 'Kansas City Chiefs',
 'LAC': 'Los Angeles Chargers',
 'LAR': 'Los Angeles Rams',
 'MIA': 'Miami Dolphins',
 'MIN': 'Minnesota Vikings',
 'NE': 'New England Patriots',
 'NO': 'New Orleans Saints',
 'NYG': 'New York Giants',
 'NYJ': 'New York Jets',
 'OAK': 'Oakland Raiders',
 'PHI': 'Philadelphia Eagles',
 'PIT': 'Pittsburgh Steelers',
 'SEA': 'Seattle Seahawks',
 'TB': 'Tampa Bay Buccaneers',
 'TEN': 'Tennessee Titans',
 'WAS': 'Washington Redskins',
 'SF': 'San Francisco 49ers'}

In [6]:
teamDict2 = {'ARI': 'Cardinals',
 'ATL': 'Falcons',
 'BAL': 'Ravens',
 'BUF': 'Bills',
 'CAR': 'Panthers',
 'CHI': 'Bears',
 'CIN': 'Bengals',
 'CLE': 'Browns',
 'DAL': 'Cowboys',
 'DEN': 'Broncos',
 'DET': 'Lions',
 'GB': 'Packers',
 'HOU': 'Texans',
 'IND': 'Colts',
 'JAX': 'Jaguars',
 'KC': 'Chiefs',
 'LAC': 'Chargers',
 'LAR': 'Rams',
 'MIA': 'Dolphins',
 'MIN': 'Vikings',
 'NE': 'Patriots',
 'NO': 'Saints',
 'NYG': 'Giants',
 'NYJ': 'Jets',
 'OAK': 'Raiders',
 'PHI': 'Eagles',
 'PIT': 'Steelers',
 'SEA': 'Seahawks',
 'TB': 'Buccaneers',
 'TEN': 'Titans',
 'WAS': 'Redskins',
 'SF': '49ers'}

In [7]:
teamDict3 = zip(teamDict.values(),teamDict2.values())
teamDict3 = dict(list(teamDict3))
teamDict3

{'Arizona Cardinals': 'Cardinals',
 'Atlanta Falcons': 'Falcons',
 'Baltimore Ravens': 'Ravens',
 'Buffalo Bills': 'Bills',
 'Carolina Panthers': 'Panthers',
 'Chicago Bears': 'Bears',
 'Cincinnati Bengals': 'Bengals',
 'Cleveland Browns': 'Browns',
 'Dallas Cowboys': 'Cowboys',
 'Denver Broncos': 'Broncos',
 'Detroit Lions': 'Lions',
 'Green Bay Packers': 'Packers',
 'Houston Texans': 'Texans',
 'Indianapolis Colts': 'Colts',
 'Jacksonville Jaguars': 'Jaguars',
 'Kansas City Chiefs': 'Chiefs',
 'Los Angeles Chargers': 'Chargers',
 'Los Angeles Rams': 'Rams',
 'Miami Dolphins': 'Dolphins',
 'Minnesota Vikings': 'Vikings',
 'New England Patriots': 'Patriots',
 'New Orleans Saints': 'Saints',
 'New York Giants': 'Giants',
 'New York Jets': 'Jets',
 'Oakland Raiders': 'Raiders',
 'Philadelphia Eagles': 'Eagles',
 'Pittsburgh Steelers': 'Steelers',
 'Seattle Seahawks': 'Seahawks',
 'Tampa Bay Buccaneers': 'Buccaneers',
 'Tennessee Titans': 'Titans',
 'Washington Redskins': 'Redskins',
 'Sa

In [8]:
# Map dictionary keys to new values so we have matching team/opponent names across the board
wr['Team'] = wr['Team'].map(teamDict2)
wr['Opponent'] = wr['Opponent'].map(teamDict2)

# qbSalaries['Team'] = qbSalaries['Team'].map(teamDict2)
# qbSalaries['Opponent'] = qbSalaries['Opponent'].map(teamDict2)


awayMatchup['team_x'] = awayMatchup['team_x'].map(teamDict3)
awayMatchup['team_y'] = awayMatchup['team_y'].map(teamDict3)
homeMatchup['team_x'] = homeMatchup['team_x'].map(teamDict3)
homeMatchup['team_y'] = homeMatchup['team_y'].map(teamDict3)

awayMatchup.head()

,gameId,date,season,week,team_x,homeAway,stadium,roof,surface,avgPf,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,201811110kan,2018-11-11,2018,10,Cardinals,away,Arrowhead Stadium,outdoors,grass,14.857143,...,0.065,37.857143,-10.285714,-13.857143,-10.600000,-16.071429,0.657143,-0.315,-0.700,14
1,201610300car,2016-10-30,2016,8,Cardinals,away,Bank of America Stadium,outdoors,grass,22.714286,...,0.036,51.285714,3.000000,4.457143,-2.228571,3.257143,-3.157143,-0.039,0.208,20
2,201612240sea,2016-12-24,2016,16,Cardinals,away,CenturyLink Field,outdoors,turf,25.857143,...,-0.114,43.857143,0.757143,-3.042857,7.171429,-14.242857,-9.314286,-0.250,-0.160,34
3,201812300sea,2018-12-30,2018,17,Cardinals,away,CenturyLink Field,outdoors,turf,13.000000,...,0.016,36.857143,-7.400000,-13.642857,-7.628571,-6.100000,-7.414286,-0.409,-0.500,24
4,201712310sea,2017-12-31,2017,17,Cardinals,away,CenturyLink Field,outdoors,turf,18.571429,...,-0.036,41.000000,-9.228571,-2.514286,-15.628571,-5.085714,-6.285714,-0.227,-0.159,26


In [9]:
#turn seasons to numeric instead of strings
wr['season'] = pd.to_numeric(wr['season'])
# qbSalaries['season'] = pd.to_numeric(qbSalaries['season'])

# Merge qb table with matchup tables, then put the two together
wrTable1 = pd.merge(wr,awayMatchup,how='inner',left_on=['Team','season','Week'],right_on=['team_x','season','week'])

wrTable2 = pd.merge(wr,homeMatchup,how='inner',left_on=['Team','season','Week'],right_on=['team_x','season','week'])

wrTableFinal = pd.concat([wrTable1,wrTable2],axis=0)

wrTableFinal.head()

# rbTableFinal = pd.merge(rbTable,rbSalaries,how='inner',left_on=['Name', 'Team', 'Position', 'Week', 'Opponent','season'],right_on=['Name', 'Team', 'Position', 'Week', 'Opponent','season'])
# rbTableFinal.columns

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,1,Wes Welker,Patriots,WR,3,Bills,20,16,80.0,217,...,-0.358,57.857143,18.985714,12.771429,11.371429,3.942857,10.485714,0.212,-0.318,31
1,4,Torrey Smith,Ravens,WR,3,Rams,8,5,62.5,152,...,0.381,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.819,1.540,37
2,468,Anquan Boldin,Ravens,WR,3,Rams,14,7,50.0,74,...,0.381,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.819,1.540,37
3,5,Miles Austin,Cowboys,WR,2,49ers,15,9,60.0,143,...,0.014,47.571429,0.385714,-0.600000,-4.885714,1.200000,-14.371429,0.030,-0.003,27
4,454,Jesse Holley,Cowboys,WR,2,49ers,3,3,100.0,96,...,0.014,47.571429,0.385714,-0.600000,-4.885714,1.200000,-14.371429,0.030,-0.003,27


In [10]:
wrs = wrTableFinal[(wrTableFinal['ReceivingTargets'] >= 1)]

In [11]:
wrs.head()

,Rank,Name,Team,Position,Week,Opponent,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,...,defDvoa,ptsMatchup,offMatchup,ovrMatchup,passMatchup,pblkMatchup,runMatchup,offDvoaMatchup,totalDvoaMatchup,pf
0,1,Wes Welker,Patriots,WR,3,Bills,20,16,80.0,217,...,-0.358,57.857143,18.985714,12.771429,11.371429,3.942857,10.485714,0.212,-0.318,31
1,4,Torrey Smith,Ravens,WR,3,Rams,8,5,62.5,152,...,0.381,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.819,1.540,37
2,468,Anquan Boldin,Ravens,WR,3,Rams,14,7,50.0,74,...,0.381,45.857143,10.585714,12.114286,9.542857,-5.900000,4.585714,0.819,1.540,37
3,5,Miles Austin,Cowboys,WR,2,49ers,15,9,60.0,143,...,0.014,47.571429,0.385714,-0.600000,-4.885714,1.200000,-14.371429,0.030,-0.003,27
4,454,Jesse Holley,Cowboys,WR,2,49ers,3,3,100.0,96,...,0.014,47.571429,0.385714,-0.600000,-4.885714,1.200000,-14.371429,0.030,-0.003,27


In [12]:
wrs = pd.merge(wrs,oddsTable,how='inner',left_on=['Team','season','Week'],right_on=['team','season','week'])

In [13]:
wrs.loc[wrs.pf > wrs.pa, 'winLose'] = 'Win'
wrs.loc[wrs.pf < wrs.pa, 'winLose'] = 'Loss'
wrs.loc[wrs.pf == wrs.pa, 'winLose'] = 'Tie'

wrs.loc[wrs.totalScore > wrs.total, 'result'] = 'over'
wrs.loc[wrs.totalScore < wrs.total, 'result'] = 'under'
wrs.loc[wrs.totalScore == wrs.total, 'result'] = 'push'

In [14]:
wrs.columns

Index(['Rank', 'Name', 'Team', 'Position', 'Week', 'Opponent',
       'ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns', 'ReceivingLong',
       'ReceivingYardsPerTarget', 'ReceivingYardsPerReception',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'Fumbles', 'FumblesLost',
       'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel', 'season',
       'gameId', 'date_x', 'week_x', 'team_x', 'homeAway', 'stadium', 'roof',
       'surface', 'avgPf', 'avgOvr_x', 'avgOff', 'avgPass', 'avgPblk',
       'avgRecv', 'avgRun', 'avgRblk', 'totalDvoa_x', 'offDvoa', 'stDvoa',
       'overUnder', 'pa', 'totalScore', 'team_y', 'avgPa', 'avgOvr_y',
       'avgDef', 'avgRdef', 'avgPrsh', 'avgCov', 'totalDvoa_y', 'defDvoa',
       'ptsMatchup', 'offMatchup', 'ovrMatchup', 'passMatchup', 'pblkMatchup',
       'runMatchup', 'offDvoaMatchup', 'totalDvoaMatchup', 'pf', 'date_y',
       'week_y', 'team', '

In [15]:
wrVars = ['Name', 'Team', 'Position','season', 'Week', 'Opponent', 'homeAway', 'surface','ReceivingTargets', 'Receptions', 'ReceptionPercentage',
       'ReceivingYards', 'ReceivingTouchdowns','defDvoa','avgCov','avgPass','passMatchup','offMatchup','offDvoaMatchup','spread','total','impliedTotal', 'winLose','pf', 'result','FantasyPointsFanDuel']

In [16]:
newWrs = wrs[wrVars]
newWrs.tail()

,Name,Team,Position,season,Week,Opponent,homeAway,surface,ReceivingTargets,Receptions,...,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,winLose,pf,result,FantasyPointsFanDuel
7255,Keelan Cole,Jaguars,WR,2018,3,Titans,home,grass,9,5,...,1.057143,1.885714,0.016,-10.0,39.5,24.75,Loss,6,under,6.5
7256,Kelvin Benjamin,Bills,WR,2018,9,Bears,home,turf,9,4,...,-23.214286,-20.942857,-0.690,10.0,38.0,14.00,Loss,9,over,6.0
7257,Marquez Valdes-Scantling,Packers,WR,2018,17,Lions,home,grass,8,3,...,6.085714,0.757143,0.276,-7.5,44.5,26.00,Loss,0,under,5.8
7258,Josh Doctson,Redskins,WR,2018,7,Cowboys,home,grass,6,3,...,-7.657143,-6.971429,-0.031,1.0,40.5,19.75,Win,20,under,5.7
7259,Seth Roberts,Raiders,WR,2018,6,Seahawks,home,grass,6,5,...,-5.514286,-4.271429,-0.037,3.0,47.5,22.25,Loss,3,under,5.6


In [17]:
corr = newWrs.corr()
corr.style.background_gradient(cmap='coolwarm')

,season,Week,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,defDvoa,avgCov,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,pf,FantasyPointsFanDuel
season,1,-0.0114168,-0.0432221,0.00666466,0.075659,-0.0271304,0.00717513,-0.0580816,0.180468,0.0581727,-0.0663813,-0.235264,-0.0670498,-0.0160573,0.0999312,0.0706757,0.0206498,-0.00324766
Week,-0.0114168,1,0.00187064,-0.0141013,-0.0253717,-0.0173888,-0.00497091,0.00953664,0.0586893,0.00222823,-0.0349125,-0.0419104,0.00995711,0.00747855,-0.0247392,-0.0203703,-0.0249408,-0.00871505
ReceivingTargets,-0.0432221,0.00187064,1,0.823717,-0.31784,0.578099,0.0204743,-0.0353,0.0217164,0.00485438,-0.00970109,0.00201791,-0.0274534,0.0435797,0.0252689,-0.021205,-0.101507,0.473775
Receptions,0.00666466,-0.0141013,0.823717,1,0.206487,0.677117,0.0562549,-0.0233695,0.0119691,0.062505,0.0424197,0.0365101,0.0240434,-0.0259768,0.0688193,0.0609089,0.0122408,0.584346
ReceptionPercentage,0.075659,-0.0253717,-0.31784,0.206487,1,0.110167,0.0448564,0.0125221,-0.0235477,0.0891097,0.0858647,0.0578542,0.0777315,-0.111619,0.0648284,0.12886,0.184628,0.126094
ReceivingYards,-0.0271304,-0.0173888,0.578099,0.677117,0.110167,1,0.169715,-0.00809734,-0.011629,0.0542593,0.0505896,0.0581325,0.0332907,-0.027399,0.0804897,0.0687911,0.121494,0.777328
ReceivingTouchdowns,0.00717513,-0.00497091,0.0204743,0.0562549,0.0448564,0.169715,1,0.0162505,-0.0407844,0.057759,0.071611,0.0639081,0.0544333,-0.0716602,0.0684155,0.0981479,0.292321,0.709193
defDvoa,-0.0580816,0.00953664,-0.0353,-0.0233695,0.0125221,-0.00809734,0.0162505,1,-0.430431,-0.0240767,0.249875,0.331133,0.607334,-0.154747,0.203558,0.244066,0.0532458,-0.000806804
avgCov,0.180468,0.0586893,0.0217164,0.0119691,-0.0235477,-0.011629,-0.0407844,-0.430431,1,0.0283054,-0.602583,-0.602358,-0.277215,0.248305,-0.100358,-0.26142,-0.105865,-0.030487
avgPass,0.0581727,0.00222823,0.00485438,0.062505,0.0891097,0.0542593,0.057759,-0.0240767,0.0283054,1,0.78068,0.523938,0.448525,-0.395814,0.377108,0.541667,0.226893,0.0770352


In [18]:
lookVars = ['Name', 'Team', 'season', 'Week', 'Opponent','homeAway', 'surface',
       'spread', 'total','result','defDvoa',
       'impliedTotal','pf', 'FantasyPointsFanDuel']

In [19]:
panth = newWrs[(newWrs['Opponent'] == 'Buccaneers') & (newWrs['Name'] == 'Julio Jones')][lookVars]
# panth.corr().style.background_gradient(cmap='coolwarm')
panth[lookVars].sort_values('FantasyPointsFanDuel',ascending=False)

,Name,Team,season,Week,Opponent,homeAway,surface,spread,total,result,defDvoa,impliedTotal,pf,FantasyPointsFanDuel
6335,Julio Jones,Falcons,2017,12,Buccaneers,home,turf,-10.0,47.5,over,0.131,28.75,34,44.8
4978,Julio Jones,Falcons,2014,3,Buccaneers,home,turf,-6.5,47.0,over,0.039,26.75,56,32.6
5481,Julio Jones,Falcons,2015,8,Buccaneers,home,turf,-8.0,48.0,under,0.014,28.00,20,26.2
3198,Julio Jones,Falcons,2018,17,Buccaneers,away,grass,-2.5,52.0,over,0.138,27.25,34,24.3
490,Julio Jones,Falcons,2012,12,Buccaneers,away,grass,-1.0,51.0,under,-0.015,26.00,24,23.7
3639,Julio Jones,Falcons,2011,17,Buccaneers,home,turf,-9.5,45.0,over,0.153,27.25,45,22.5
2294,Julio Jones,Falcons,2016,9,Buccaneers,away,grass,-3.5,50.5,over,-0.004,27.00,43,21.1
6957,Julio Jones,Falcons,2018,6,Buccaneers,home,turf,-3.0,57.5,over,0.289,30.25,34,19.4
1527,Julio Jones,Falcons,2014,10,Buccaneers,away,grass,-3.0,47.0,under,0.128,25.00,27,15.9
133,Julio Jones,Falcons,2011,3,Buccaneers,away,grass,1.0,45.0,under,0.224,22.00,13,13.6


In [34]:
th = newWrs[(newWrs['Name'] == 'Alshon Jeffery') & (newWrs['Team'] == 'Eagles') & (newWrs['ReceivingTargets'] >= 6)]
# th[wrVars].corr().style.background_gradient(cmap='coolwarm')
th[lookVars].sort_values('FantasyPointsFanDuel',ascending=False)

,Name,Team,season,Week,Opponent,homeAway,surface,spread,total,result,defDvoa,impliedTotal,pf,FantasyPointsFanDuel
6396,Alshon Jeffery,Eagles,2017,9,Broncos,home,grass,-7.0,41.0,over,-0.130,24.00,51,23.4
3228,Alshon Jeffery,Eagles,2018,4,Titans,away,grass,-2.5,41.5,over,0.159,22.00,23,20.5
3237,Alshon Jeffery,Eagles,2018,15,Rams,away,grass,13.5,52.5,over,-0.014,19.50,30,20.0
2760,Alshon Jeffery,Eagles,2017,2,Chiefs,away,grass,4.0,46.5,over,-0.008,21.25,20,18.7
6977,Alshon Jeffery,Eagles,2018,7,Panthers,home,grass,-4.5,45.0,under,0.089,24.75,17,18.3
2806,Alshon Jeffery,Eagles,2017,11,Cowboys,away,turf,-6.0,48.0,under,0.084,27.00,37,16.7
6546,Alshon Jeffery,Eagles,2017,8,49ers,home,grass,-12.5,45.0,under,0.093,28.75,33,15.2
3407,Alshon Jeffery,Eagles,2018,14,Cowboys,away,turf,3.0,45.5,over,-0.015,21.25,23,14.0
2907,Alshon Jeffery,Eagles,2017,14,Rams,away,grass,-1.0,47.5,over,-0.134,24.25,43,13.7
6510,Alshon Jeffery,Eagles,2017,12,Bears,home,grass,-13.5,43.0,under,-0.073,28.25,31,13.7


In [35]:
np.mean(th['FantasyPointsFanDuel'])

13.75294117647059

In [47]:
class WR:
    
    def __init__(self,name,table):
        self.name = name
        self.table = table[table['Name'] == name]
        
    def get_table(self):
        return self.table
    
    def corrMatrix(self,features):
        correlations = self.table[features].corr()
        return correlations.style.background_gradient(cmap='coolwarm')
    
    def scatter(self,var1,var2):
        plt.scatter(self.table[var1],self.table[var2])
        plt.title(self.name)
        plt.xlabel(var1)
        plt.ylabel(var2)
        plt.show()
        
    def impliedTot(self, value):
        return self.table[self.table['impliedTotal'] >= value][wrVars]

In [48]:
ka = WR('Julio Jones',newWrs)
ka.corrMatrix(wrVars)

,season,Week,ReceivingTargets,Receptions,ReceptionPercentage,ReceivingYards,ReceivingTouchdowns,defDvoa,avgCov,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,FantasyPointsFanDuel
season,1,-0.00191025,0.157167,0.148875,-0.0695267,0.112653,-0.150591,-0.0750559,0.18085,0.105007,-0.0901927,-0.103349,-0.0672435,0.00141104,0.288441,0.223014,0.00511734
Week,-0.00191025,1,-0.0908816,-0.156021,-0.126184,-0.0856558,0.0844819,0.112585,0.111264,-0.0137742,-0.100017,-0.0839412,0.0232396,-0.00954431,0.0396993,0.0396376,-0.0365227
ReceivingTargets,0.157167,-0.0908816,1,0.815398,-0.291135,0.560989,0.0300959,-0.176136,0.145924,-0.0137794,-0.128738,-0.0877522,-0.0924595,0.176725,0.0895229,-0.0926744,0.458388
Receptions,0.148875,-0.156021,0.815398,1,0.284005,0.751338,0.202675,-0.0923144,0.0983044,-0.0811339,-0.127547,-0.0614138,-0.0722075,0.232817,0.161912,-0.087891,0.674311
ReceptionPercentage,-0.0695267,-0.126184,-0.291135,0.284005,1,0.300221,0.320861,0.142494,-0.0691104,-0.13953,-0.0220056,0.0191891,0.00949425,0.112678,0.0624027,-0.0549476,0.367524
ReceivingYards,0.112653,-0.0856558,0.560989,0.751338,0.300221,1,0.300671,-0.126633,0.20257,-0.0345598,-0.18748,-0.16469,-0.0880395,0.235659,0.0954742,-0.142167,0.852365
ReceivingTouchdowns,-0.150591,0.0844819,0.0300959,0.202675,0.320861,0.300671,1,0.0927024,0.0511392,-0.1758,-0.142248,-0.0463106,0.0642348,0.0204394,-0.0702337,-0.0733883,0.721911
defDvoa,-0.0750559,0.112585,-0.176136,-0.0923144,0.142494,-0.126633,0.0927024,1,-0.448059,-0.230764,0.240154,0.254064,0.668837,-0.192004,0.286548,0.399161,-0.044926
avgCov,0.18085,0.111264,0.145924,0.0983044,-0.0691104,0.20257,0.0511392,-0.448059,1,0.00988804,-0.822967,-0.734265,-0.385381,0.295472,-0.147323,-0.38591,0.143515
avgPass,0.105007,-0.0137742,-0.0137794,-0.0811339,-0.13953,-0.0345598,-0.1758,-0.230764,0.00988804,1,0.559924,0.385939,0.178821,-0.172374,0.140829,0.267815,-0.114227


In [54]:
allen = ka.get_table()
allen[(allen['surface'] == 'turf')]

,Name,Team,Position,season,Week,Opponent,homeAway,surface,ReceivingTargets,Receptions,...,avgPass,passMatchup,offMatchup,offDvoaMatchup,spread,total,impliedTotal,winLose,result,FantasyPointsFanDuel
29,Julio Jones,Falcons,WR,2011,9,Colts,away,turf,4,3,...,66.514286,11.285714,7.071429,0.324,-6.5,45.0,25.75,Win,under,29.9
92,Julio Jones,Falcons,WR,2011,16,Saints,away,turf,14,8,...,73.557143,18.757143,18.942857,0.271,7.0,52.5,22.75,Loss,over,20.8
149,Julio Jones,Falcons,WR,2011,4,Seahawks,away,turf,17,11,...,68.414286,9.528571,-1.257143,0.009,-6.0,39.0,22.50,Win,over,18.2
479,Julio Jones,Falcons,WR,2012,16,Lions,away,turf,11,7,...,68.357143,4.542857,8.200000,0.132,-3.5,50.0,26.75,Win,under,15.9
729,Julio Jones,Falcons,WR,2012,10,Saints,away,turf,5,4,...,76.385714,22.814286,19.471429,0.312,-2.0,53.5,27.75,Loss,over,9.5
1210,Julio Jones,Falcons,WR,2013,1,Saints,away,turf,9,7,...,75.014286,14.142857,17.457143,0.140,3.5,54.5,25.50,Loss,under,15.1
1539,Julio Jones,Falcons,WR,2014,7,Ravens,away,turf,8,5,...,74.385714,16.200000,6.028571,0.071,6.5,50.0,21.75,Loss,under,8.1
1605,Julio Jones,Falcons,WR,2014,2,Bengals,away,turf,13,7,...,70.142857,3.485714,3.557143,0.356,5.5,49.0,21.75,Loss,under,18.3
1656,Julio Jones,Falcons,WR,2014,5,Giants,away,turf,16,11,...,71.700000,2.957143,0.800000,0.155,4.5,50.5,23.00,Loss,under,16.0
1666,Julio Jones,Falcons,WR,2014,4,Vikings,away,turf,8,6,...,71.471429,16.714286,8.000000,0.268,-5.0,47.5,26.25,Loss,over,11.2


In [63]:
np.mean(allen[(allen['homeAway'] == 'away')]['FantasyPointsFanDuel'])

17.11132075471698